In [ ]:
# В РЕЖИМІ РЕАЛЬНОГО ЧАСУ
import pyaudio
import os
import struct
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft
import time
from tkinter import TclError
import math

def nota(x):
# перелік нот в порядку збільшення її частоти
    noty=['Сі-0','До','До#','Ре','Ре#','Мі','Фа','Фа#','Соль','Соль#','Ля','Ля#','Ci', 'До-2', 'До#-2','Ре-2','Ре#-2','Мі-2','Фа-2','Фа#-2','Соль-2','Соль#-2','Ля-2','Ля#-2','Ci-2', 'До-3', 'До#-3', 'Ре-3', 'Ре#-3']
# коефіцієнт рівномірно-темперованої шкали; у скільки раз ноти відрізняються одна від одної
    p=2**(1/12)

# створюємо масив частот і їх назв (назв нот)
    muz=[]
    for i in range(len(noty)):
        muz.append([440*(p**i)/(p**10), noty[i]])
    
    riz=[]
    for i in range(len(muz)):
        riz.append(abs(muz[i][0]-x))
    if min(riz)>60:
        s='--'
    else:
        s=muz[riz.index(min(riz))][1]
    return s

%matplotlib tk

CHUNK = 1024 * 8           
FORMAT = pyaudio.paInt16    
CHANNELS = 1                
RATE = 44100              

# створюємо фігуру, координатні осі
fig, (ax1, ax2) = plt.subplots(2, figsize=(10, 3))

p = pyaudio.PyAudio()
 
stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    output=True,
    frames_per_buffer=CHUNK)
 
x = np.arange(0, 2 * CHUNK, 2)  #від 0 до 2*CHUNK з кроком 2    
xf = np.linspace(0, RATE, CHUNK)  #від 0 до RATE розмістити CHUNK точок 

print(len(x), len(xf),'------')

#тиск від часу 
#заповнюємо випадковими значеннями масиви розміром CHUNK
line, = ax1.plot(x, np.random.rand(CHUNK), '-', lw=2)
# plot повертає одноелементный список, який распаковується в line
#або так [line]=...
# об'єкт line створює посилання на дані графіка ax1

#АЧХ, логарифмічна вісь Ох
line_fft, = ax2.semilogx(xf, np.random.rand(CHUNK), '-', lw=2)

ax1.set_title('Графік зміни тиску від часу')
ax1.set_xlabel('фрагмент')
ax1.set_ylabel('значення')

#обмеження по осі Ox, Оу для першого графіка
ax1.set_ylim(-15000, 15000)
ax1.set_xlim(0, 512)

#підписуємо основні точки осі першого графіка
plt.setp(ax1, xticks=[0, 256, 512], yticks=[-10000, 0, 10000])
# ax1.set_yscale('log')

#обмеження по осі Ox другого графіка від 1 до 20000
ax2.set_xlim(1, RATE / 2)
ax2.set_ylim(0, 3000)
# ax2.set_yscale('log')

print('розпочався потік')

# кількість проаналізованих фрагментів по CHUNK
frame_count = 0
s=0

annotation = ax2.annotate('', xy=(0,0)) 
annotation2 = ax2.annotate('', xy=(0,0)) 

i=0
while True:
    i=i+1
    
    stream.start_stream()
    data = np.frombuffer(stream.read(CHUNK),dtype=np.int16)
    stream.stop_stream()
    data_int =data
    data_np = data

    #оновити дані
    line.set_ydata(data_np)
    yf = fft(data_int)
    line_fft.set_ydata(np.abs(yf[0:CHUNK])  / (0.5 * CHUNK))
    # y1.append(abs(y_1[i])/(L/2))
    # L=CHUNK
    yf=yf[0:4096]
    yf2=[]
    for k in range(len(yf)):
        yf2.append(abs(yf[k])/(0.5 * CHUNK))
    yf1=[]
    for k in range(1,len(yf2)-1):
        if yf2[k]>yf2[k-1] and yf2[k]>yf2[k+1]:
            yf1.append([yf2[k],xf[k]])
    yf1=sorted(yf1, key=lambda yf1: yf1[0], reverse=True)
    
    #анотація
    #round
    new_x=yf1[0][1]
    new_y=yf1[0][0]
    annotation.set_position((new_x,new_y))
    annotation.xy = (new_x,new_y)
    s='['+str(round(new_x))+'Гц'+'; '+str(round(new_y))+']; '+nota(new_x)
    annotation.set_text(s)
    print('N=',i,s)
    
 #пробуємо візуалізувати дані
    try:
         # перерисувати фігуру
        #нарисувати полотно
        fig.canvas.draw()
        # очистити всі очікувані події графічного інтерфейсу, перефарбувавши екран, якщо потрібно
        fig.canvas.flush_events()
        
        #збільшити frame_count на 1
        frame_count += 1
   
    # щоб уникнути помилок при відсутності даних
    # tkinter.TclError
    except TclError:
        print('потік зупинився')

        # відсутність операцій pass 
        # frame_rate = frame_count / (time.time() - start_time)
       
        # print('середня частота кадрів = {:.0f} FPS'.format(frame_rate))
        # виходимо з циклу 
        break
        
        # matplotlib.org

8192 8192 ------
розпочався потік
N= 1 [16Гц; 28]; --
N= 2 [11Гц; 70]; --
N= 3 [5Гц; 64]; --
N= 4 [32Гц; 25]; --
N= 5 [226Гц; 195]; Сі-0
N= 6 [5Гц; 105]; --
N= 7 [5Гц; 74]; --
N= 8 [5Гц; 76]; --
N= 9 [11Гц; 79]; --
N= 10 [5Гц; 130]; --
N= 11 [48Гц; 18]; --
